In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
BD = "bookings_data"
B = "bookings"
CD = "customer_data"
HD = "hotels_data"
PD = "payments_data"
SS = "sample_submission_5"
TD = "train_data"

base_path = r"Dataset/Processed_data/"

def load_from_csv(filename):
    file_path = base_path + filename + ".csv"
    return pd.read_csv(file_path)

In [3]:
bookings_data = load_from_csv(BD)
bookings = load_from_csv(B)
customer_data = load_from_csv(CD)
hotels_data = load_from_csv(HD)
payments_data = load_from_csv(PD)
sample_submission = load_from_csv(SS)
train_data = load_from_csv(TD)

bookings['booking_create_timestamp'] = pd.to_datetime(bookings['booking_create_timestamp'])
bookings['booking_approved_at'] = pd.to_datetime(bookings['booking_approved_at'])
bookings['booking_checkin_customer_date'] = pd.to_datetime(bookings['booking_checkin_customer_date'])
bookings_data['booking_expiry_date'] = pd.to_datetime(bookings_data['booking_expiry_date'])

In [39]:
# bookings_data.describe()
# bookings.describe()
# customer_data.describe()
# hotels_data.describe()
payments_data.describe()
# sample_submission.describe()
# train_data.describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100380
std,0.706584,2.687051,217.494064
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790000
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837500
max,29.000000,24.000000,13664.080000


In [44]:
bookings_data['booking_id'].unique()


array(['242fe8c5a6d1ba2dd792cb1621400010',
       'f77f2f0320c557190d7a144bdd300018',
       '98224ef6ca0657da4fc703e000229ec3', ...,
       '5a9662cd70adb13d4a31832dfffce470',
       'ffabc95dfada21779c9644ffffe18544',
       'cc87c801fd61db3f6244fffe41c64501'], dtype=object)

In [4]:
def get_hash(val_list):
    hashmap = {}
    for i,val in enumerate(val_list):
        hashmap[val] = i
    return hashmap

In [5]:
# payments_data_modified = pd.DataFrame(data=[], columns=['booking_id', 'payments_made', 'payments_value'])
unique_ids = payments_data['booking_id'].unique()
hash_val = get_hash(unique_ids)
counts = np.zeros(unique_ids.shape)
total_installments = np.zeros(unique_ids.shape)
value_sum = np.zeros(unique_ids.shape)
for i in range(payments_data.shape[0]):
    booking_id = payments_data.at[i, 'booking_id']
    installments = payments_data.at[i, 'payment_installments']
    payment_value = payments_data.at[i, 'payment_value']
    counts[hash_val[booking_id]] += 1
    total_installments[hash_val[booking_id]] += installments
    value_sum[hash_val[booking_id]] += payment_value
payments_data_modified = []
for booking_id in unique_ids:
    payments_made =  counts[hash_val[booking_id]]
    payments_installments = total_installments[hash_val[booking_id]]
    payments_value = value_sum[hash_val[booking_id]]
    payments_data_modified.append([booking_id, payments_made, payments_installments, payments_value])
payments_data_modified = pd.DataFrame(data=payments_data_modified, columns=['booking_id', 'payments_made', 'payments_installments', 'payments_value'])
payments_data_modified.describe(include='all')

,booking_id,payments_made,payments_installments,payments_value
count,99440,99440.000000,99440.000000,99440.000000
unique,99440,NaN,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,NaN,NaN
freq,1,NaN,NaN,NaN
mean,NaN,1.044710,2.980923,160.990267
std,NaN,0.381166,2.741810,221.951257
min,NaN,1.000000,0.000000,0.000000
25%,NaN,1.000000,1.000000,62.010000
50%,NaN,1.000000,2.000000,105.290000
75%,NaN,1.000000,4.000000,176.970000


In [6]:
# Has NaNs!!!
unique_ids = hotels_data['hotel_id'].unique()
hotel_id_hash = get_hash(unique_ids)
hotels_data_modified = []
for i in range(hotels_data.shape[0]):
    hotel_id = hotel_id_hash[hotels_data.at[i, 'hotel_id']]
    category = hotels_data.at[i, 'hotel_category']
    name_length = hotels_data.at[i, 'hotel_name_length']
    description_length = hotels_data.at[i, 'hotel_description_length']
    photos_qty = hotels_data.at[i, 'hotel_photos_qty']
    hotels_data_modified.append([hotel_id, category, name_length, description_length, photos_qty])
hotels_data_modified = pd.DataFrame(data=hotels_data_modified, columns=[
    'hotel_id', 'hotel_category', 'hotel_name_length', 'hotel_description_length', 'hotel_photos_qty'])
hotels_data_modified.fillna(0, inplace=True)
hotels_data_modified.describe(include='all')

,hotel_id,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,32951.000000,32951.000000,32951.000000,32951.000000,32951.000000
mean,16475.000000,29.715790,47.579527,757.213104,2.148463
std,9512.278697,23.870388,12.071951,637.745057,1.745732
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8237.500000,10.000000,41.000000,326.000000,1.000000
50%,16475.000000,28.000000,51.000000,584.000000,1.000000
75%,24712.500000,41.000000,57.000000,961.000000,3.000000
max,32950.000000,73.000000,76.000000,3992.000000,20.000000


In [7]:
unique_unique_ids = customer_data['customer_unique_id'].unique()
unique_id_hash = get_hash(unique_unique_ids)
unique_countries = customer_data['country'].unique()
country_hash = get_hash(unique_countries)
customer_data_modified = []
for i in range(customer_data.shape[0]):
    customer_id = customer_data.at[i, 'customer_id']
    unique_id_num = unique_id_hash[customer_data.at[i, 'customer_unique_id']]
    country_num = country_hash[customer_data.at[i, 'country']]
    customer_data_modified.append([customer_id, unique_id_num, country_num])
customer_data_modified = pd.DataFrame(data=customer_data_modified, columns=['customer_id', 'unique_id_num', 'country_num'])
customer_data_modified.describe(include='all')


,customer_id,unique_id_num,country_num
count,99441,99441.00000,99441.000000
unique,99441,NaN,NaN
top,1fbc88172c00ba8bc706b8999e2fba1a,NaN,NaN
freq,1,NaN,NaN
mean,NaN,47470.47591,4.001428
std,NaN,27757.61303,2.578318
min,NaN,0.00000,0.000000
25%,NaN,23353.00000,2.000000
50%,NaN,47201.00000,4.000000
75%,NaN,71453.00000,6.000000


In [8]:
# Missing: dates besides created
unique_booking_status = bookings['booking_status'].unique()
status_hash = get_hash(unique_booking_status)
bookings_modified = []
for i in range(bookings.shape[0]):
    booking_id = bookings.at[i, 'booking_id']
    customer_id = bookings.at[i, 'customer_id']
    booking_status_num = status_hash[bookings.at[i, 'booking_status']]
    create_date = bookings.at[i, 'booking_create_timestamp']
    approved_date = bookings.at[i, 'booking_approved_at']
    checkin_date = bookings.at[i, 'booking_checkin_customer_date']
    approval_time = (approved_date-create_date).total_seconds()/60
    checkin_time = (checkin_date-create_date).total_seconds()/1440
    bookings_modified.append([booking_id, customer_id, booking_status_num, create_date, approval_time, checkin_time])
bookings_modified = pd.DataFrame(data=bookings_modified, columns=[
    'booking_id', 'customer_id', 'booking_status_num', 'booking_create_timestamp', 'booking_approval_time', 'booking_checkin_time'])
bookings_modified.fillna(-1, inplace=True)
bookings_modified.describe(include='all')

/tmp/ipykernel_52512/318350241.py:18: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  bookings_modified.describe(include='all')


,booking_id,customer_id,booking_status_num,booking_create_timestamp,booking_approval_time,booking_checkin_time
count,99441,99441,99441.000000,99441,99441.000000,99441.000000
unique,99441,99441,NaN,98875,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,51297304e76186b10a928d9ef432eb62,NaN,2008-04-13 10:31:14,NaN,NaN
freq,1,1,NaN,3,NaN,NaN
first,NaN,NaN,NaN,2006-09-07 20:58:19,NaN,NaN
last,NaN,NaN,NaN,2008-10-19 17:13:18,NaN,NaN
mean,NaN,NaN,0.090868,NaN,624.138193,731.024797
std,NaN,NaN,0.569725,NaN,1561.224567,578.598782
min,NaN,NaN,0.000000,NaN,-1.000000,-1.000000
25%,NaN,NaN,0.000000,NaN,12.883333,377.969444


In [9]:
unique_ids = bookings_data['booking_id'].unique()
hash_val = get_hash(unique_ids)
unique_agents = bookings_data['seller_agent_id'].unique()
agent_hash = get_hash(unique_agents)
counts = np.zeros(unique_ids.shape)
hotel_ids = np.zeros(unique_ids.shape).tolist()
seller_agent_ids = np.zeros(unique_ids.shape).tolist()
booking_expiry_dates = np.zeros(unique_ids.shape).tolist()
prices = np.zeros(unique_ids.shape).tolist()
agent_feess = np.zeros(unique_ids.shape).tolist()
for i in range(bookings_data.shape[0]):
    booking_id = bookings_data.at[i, 'booking_id']
    counts[hash_val[booking_id]] += 1
    hotel_ids[hash_val[booking_id]] = hotel_id_hash[bookings_data.at[i, 'hotel_id']]
    seller_agent_ids[hash_val[booking_id]] = agent_hash[bookings_data.at[i, 'seller_agent_id']]
    booking_expiry_dates[hash_val[booking_id]] = bookings_data.at[i, 'booking_expiry_date']
    prices[hash_val[booking_id]] = bookings_data.at[i, 'price']
    agent_feess[hash_val[booking_id]] = bookings_data.at[i, 'agent_fees']
bookings_data_modified = []
for booking_id in unique_ids:
    sub_requests = counts[hash_val[booking_id]]
    hotel_id = hotel_ids[hash_val[booking_id]]
    seller_agent_id = seller_agent_ids[hash_val[booking_id]]
    booking_expiry_date = booking_expiry_dates[hash_val[booking_id]]
    price = prices[hash_val[booking_id]]
    agent_fees = agent_feess[hash_val[booking_id]]
    bookings_data_modified.append([booking_id, sub_requests, hotel_id, seller_agent_id, booking_expiry_date, price, agent_fees])
bookings_data_modified =  pd.DataFrame(data=bookings_data_modified, columns=[
    'booking_id', 'sub_requests', 'hotel_id', 'seller_agent_id_num', 'booking_expiry_date', 'price', 'agent_fees'
])
bookings_data_modified.describe(include='all')

/tmp/ipykernel_52512/2095638059.py:31: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  bookings_data_modified.describe(include='all')


,booking_id,sub_requests,hotel_id,seller_agent_id_num,booking_expiry_date,price,agent_fees
count,98666,98666.000000,98666.000000,98666.000000,98666,98666.000000,98666.000000
unique,98666,NaN,NaN,NaN,93009,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,NaN,NaN,2018-06-11 03:31:04,NaN,NaN
freq,1,NaN,NaN,NaN,6,NaN,NaN
first,NaN,NaN,NaN,NaN,2016-09-19 00:15:34,NaN,NaN
last,NaN,NaN,NaN,NaN,2020-04-09 22:35:08,NaN,NaN
mean,NaN,1.141731,16536.404293,471.238157,NaN,125.883144,20.179411
std,NaN,0.538452,9578.771521,555.651909,NaN,191.166764,15.855944
min,NaN,1.000000,0.000000,0.000000,NaN,0.850000,0.000000
25%,NaN,1.000000,8290.000000,88.000000,NaN,41.492500,13.280000


In [10]:
train_data_full = pd.merge(left=train_data, right=bookings_modified, how='left', on='booking_id')
train_data_full = pd.merge(left=train_data_full, right=bookings_data_modified, how='left', on='booking_id')
train_data_full = pd.merge(left=train_data_full, right=customer_data_modified, how='left', on='customer_id')
train_data_full = pd.merge(left=train_data_full, right=hotels_data_modified, how='left', on='hotel_id')
train_data_full = pd.merge(left=train_data_full, right=payments_data_modified, how='left', on='booking_id')
# train_data_full.describe(include='all')
print(train_data_full.shape[0])
booking_expiry = []
unique_ids = train_data_full['booking_id'].unique()
id_hash = get_hash(unique_ids)
expiry_times = np.zeros(unique_ids.shape)
for i in range(train_data_full.shape[0]):
    booking_id = train_data_full.at[i, 'booking_id']
    expiry_date = train_data_full.at[i, 'booking_expiry_date']
    create_date = train_data_full.at[i, 'booking_create_timestamp']
    expiry_time = (expiry_date-create_date).total_seconds()/1440
    expiry_times[id_hash[booking_id]] = expiry_time
for booking_id in unique_ids:
    expiry_time = expiry_times[id_hash[booking_id]]
    booking_expiry.append([booking_id, expiry_time])
booking_expiry = pd.DataFrame(data=booking_expiry, columns=['booking_id', 'booking_expiry_time'])
print(booking_expiry.shape[0])
train_data_full = pd.merge(left=train_data_full, right=booking_expiry, how='left', on='booking_id')
train_data_full.drop(labels=['booking_create_timestamp', 'booking_expiry_date'], axis=1, inplace=True)
train_data_full.fillna(-1, inplace=True)
train_data_full.describe(include='all')
# train_data_full.columns.shape

50000
49868


,booking_id,rating_score,customer_id,booking_status_num,booking_approval_time,booking_checkin_time,sub_requests,hotel_id,seller_agent_id_num,price,...,unique_id_num,country_num,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,payments_made,payments_installments,payments_value,booking_expiry_time
count,50000,50000.000000,50000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
unique,49868,NaN,49868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,a9999ce368f218a407141c88b1d1b157,NaN,48c4fd30f19f55453ec4aae0fb7b01c5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,3,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,4.088940,NaN,0.089240,624.366724,729.685673,1.125240,16420.18836,468.104980,123.497017,...,47485.093260,4.001740,28.171160,47.788620,775.244800,2.191220,1.046380,2.974380,159.889098,217691.793446
std,NaN,1.346798,NaN,0.565157,1280.909201,582.049564,0.559394,9654.48287,555.874929,187.074813,...,27774.566666,2.576368,22.782656,12.220465,653.745044,1.768535,0.411767,2.738539,223.782603,19281.900447
min,NaN,1.000000,NaN,0.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,...,1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,0.000000,-1.000000
25%,NaN,4.000000,NaN,0.000000,12.883333,376.710069,1.000000,8142.00000,87.000000,39.990000,...,23406.500000,2.000000,10.000000,42.000000,340.000000,1.000000,1.000000,1.000000,61.800000,219301.013194
50%,NaN,5.000000,NaN,0.000000,20.650000,602.815625,1.000000,16235.00000,259.000000,78.000000,...,47167.000000,4.000000,26.000000,51.000000,596.000000,1.000000,1.000000,2.000000,104.995000,219361.400347
75%,NaN,5.000000,NaN,0.000000,868.795833,922.593403,1.000000,24884.00000,644.000000,139.000000,...,71466.250000,6.000000,38.000000,57.000000,982.000000,3.000000,1.000000,4.000000,176.000000,219428.191667


In [11]:
test_data = sample_submission['booking_id']
test_data = pd.merge(left=test_data, right=bookings_modified, how='left', on='booking_id')
test_data = pd.merge(left=test_data, right=bookings_data_modified, how='left', on='booking_id')
test_data = pd.merge(left=test_data, right=customer_data_modified, how='left', on='customer_id')
test_data = pd.merge(left=test_data, right=hotels_data_modified, how='left', on='hotel_id')
test_data = pd.merge(left=test_data, right=payments_data_modified, how='left', on='booking_id')
# test_data.describe(include='all')
print(test_data.shape[0])
booking_expiry = []
unique_ids = test_data['booking_id'].unique()
id_hash = get_hash(unique_ids)
expiry_times = np.zeros(unique_ids.shape)
for i in range(test_data.shape[0]):
    booking_id = test_data.at[i, 'booking_id']
    expiry_date = test_data.at[i, 'booking_expiry_date']
    create_date = test_data.at[i, 'booking_create_timestamp']
    expiry_time = (expiry_date-create_date).total_seconds()/1440
    expiry_times[id_hash[booking_id]] = expiry_time
for booking_id in unique_ids:
    expiry_time = expiry_times[id_hash[booking_id]]
    booking_expiry.append([booking_id, expiry_time])
booking_expiry = pd.DataFrame(data=booking_expiry, columns=['booking_id', 'booking_expiry_time'])
print(booking_expiry.shape[0])
test_data = pd.merge(left=test_data, right=booking_expiry, how='left', on='booking_id')
test_data.drop(labels=['booking_create_timestamp', 'booking_expiry_date'], axis=1, inplace=True)
test_data.fillna(-1, inplace=True)
test_data.describe(include='all')
# test_data.columns.shape

49079
49079


,booking_id,customer_id,booking_status_num,booking_approval_time,booking_checkin_time,sub_requests,hotel_id,seller_agent_id_num,price,agent_fees,unique_id_num,country_num,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,payments_made,payments_installments,payments_value,booking_expiry_time
count,49079,49079,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000,49079.000000
unique,49079,49079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,796b98fdf73dbeba33a548910a1c6147,531e4dc76bd6d853e68a2e7ef9053dea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.087043,623.359821,729.529599,1.124310,16397.959188,466.100063,125.747902,20.005075,47351.441757,4.001019,28.298050,47.748283,776.267528,2.190224,1.043399,2.988957,161.357086,217751.508575
std,NaN,NaN,0.558931,1792.413445,564.162745,0.569377,9647.682423,553.531153,191.303671,16.001487,27737.081265,2.580542,22.892116,12.242457,659.931297,1.765860,0.353577,2.746373,216.918271,18929.982913
min,NaN,NaN,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,NaN,NaN,0.000000,12.883333,379.818056,1.000000,8095.500000,87.000000,39.990000,13.120000,23169.500000,2.000000,9.000000,42.000000,340.000000,1.000000,1.000000,1.000000,62.055000,219301.028472
50%,NaN,NaN,0.000000,20.450000,600.942361,1.000000,16151.000000,258.000000,78.990000,16.320000,47091.000000,4.000000,27.000000,51.000000,595.000000,1.000000,1.000000,2.000000,105.380000,219361.396528
75%,NaN,NaN,0.000000,876.383333,918.825000,1.000000,24851.500000,642.000000,139.900000,21.150000,71316.500000,6.000000,38.000000,57.000000,982.000000,3.000000,1.000000,4.000000,177.270000,219429.476389


In [44]:
train_data_full.columns

Index(['booking_id', 'rating_score', 'customer_id', 'booking_status_num',
       'booking_approval_time', 'booking_checkin_time', 'sub_requests',
       'hotel_id', 'seller_agent_id_num', 'price', 'agent_fees',
       'unique_id_num', 'country_num', 'hotel_category', 'hotel_name_length',
       'hotel_description_length', 'hotel_photos_qty', 'payments_made',
       'payments_installments', 'payments_value', 'booking_expiry_time'],
      dtype='object')

In [12]:
# from sklearn.ensemble import RandomForestClassifier as rf
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
train_labels = train_data_full['rating_score']
train_data_full.drop(labels=['rating_score', 'booking_id', 'customer_id'], axis=1, inplace=True)
# X_train, X_val, Y_train, Y_val = train_test_split(train_data_full, train_labels, test_size=0.2)
# rfclf = rf()
# rfclf.fit(X_train, Y_train)
# train_pred = rfclf.predict(X_val)
# print(np.mean(np.subtract(Y_val, train_pred)**2))

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(train_data_full, train_labels, test_size=0.2)

In [62]:
from sklearn.linear_model import LogisticRegression
lrclf = LogisticRegression()
lrclf.fit(X_train, Y_train)
train_pred = lrclf.predict(X_val)
print(np.mean(np.subtract(Y_val, train_pred)**2))

2.3915


/home/balakrishnan/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [63]:
from sklearn.svm import SVC
svclf = SVC()
lrclf.fit(X_train, Y_train)
train_pred = lrclf.predict(X_val)
print(np.mean(np.subtract(Y_val, train_pred)**2))

2.3915


/home/balakrishnan/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [64]:
from sklearn.tree import DecisionTreeClassifier
dtclf = DecisionTreeClassifier()
dtclf.fit(X_train, Y_train)
train_pred = dtclf.predict(X_val)
print(np.mean(np.subtract(Y_val, train_pred)**2))

2.7181


In [65]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier()
rfclf.fit(X_train, Y_train)
train_pred = rfclf.predict(X_val)
print(np.mean(np.subtract(Y_val, train_pred)**2))

2.0107


In [66]:
from sklearn.ensemble import AdaBoostClassifier
abclf = AdaBoostClassifier()
abclf.fit(X_train, Y_train)
train_pred = abclf.predict(X_val)
print(np.mean(np.subtract(Y_val, train_pred)**2))

2.0411


In [68]:
import xgboost as xgb
xgbclf = xgb.XGBClassifier()
Y_train -= 1
xgbclf.fit(X_train, Y_train)
train_pred = xgbclf.predict(X_val)
train_pred += 1
print(np.mean(np.subtract(Y_val, train_pred)**2))

2.0186


In [71]:
import catboost as cb
cbclf = cb.CatBoostClassifier()
cbclf.fit(X_train, Y_train)
train_pred = np.reshape(cbclf.predict(X_val), Y_val.shape)
print(np.mean(np.subtract(Y_val, train_pred)**2))

Learning rate set to 0.095505
0:	learn: 1.5042837	total: 10.4ms	remaining: 10.4s
1:	learn: 1.4289573	total: 20.4ms	remaining: 10.2s
2:	learn: 1.3707833	total: 30.3ms	remaining: 10.1s
3:	learn: 1.3262201	total: 41ms	remaining: 10.2s
4:	learn: 1.2895666	total: 52.3ms	remaining: 10.4s
5:	learn: 1.2588279	total: 62.5ms	remaining: 10.4s
6:	learn: 1.2345701	total: 72.1ms	remaining: 10.2s
7:	learn: 1.2144706	total: 83ms	remaining: 10.3s
8:	learn: 1.1981205	total: 92.7ms	remaining: 10.2s
9:	learn: 1.1831850	total: 107ms	remaining: 10.5s
10:	learn: 1.1711519	total: 114ms	remaining: 10.3s
11:	learn: 1.1620376	total: 124ms	remaining: 10.2s
12:	learn: 1.1534998	total: 133ms	remaining: 10.1s
13:	learn: 1.1448213	total: 142ms	remaining: 9.98s
14:	learn: 1.1370624	total: 150ms	remaining: 9.87s
15:	learn: 1.1305812	total: 160ms	remaining: 9.85s
16:	learn: 1.1254134	total: 169ms	remaining: 9.75s
17:	learn: 1.1209517	total: 178ms	remaining: 9.69s
18:	learn: 1.1174854	total: 187ms	remaining: 9.64s
19:	le

In [72]:
import lightgbm as lgb
lgbclf = lgb.LGBMClassifier()
lgbclf.fit(X_train, Y_train)
train_pred = np.reshape(lgbclf.predict(X_val), Y_val.shape)
print(np.mean(np.subtract(Y_val, train_pred)**2))

1.8333


In [16]:
test_ids = test_data['booking_id']
test_data.drop(labels=['booking_id', 'customer_id'], axis=1, inplace=True)

In [19]:
import lightgbm as lgb
lgbclf = lgb.LGBMClassifier()
lgbclf.fit(train_data_full, train_labels)
test_pred = np.reshape(lgbclf.predict(test_data), test_ids.shape)
test_pred = pd.DataFrame(data=test_pred, columns=['rating_score'])
test_sub = pd.concat([test_ids, test_pred], axis=1)
test_sub.describe(include='all')

,booking_id,rating_score
count,49079,49079.000000
unique,49079,NaN
top,796b98fdf73dbeba33a548910a1c6147,NaN
freq,1,NaN
mean,NaN,4.671570
std,NaN,1.092348
min,NaN,1.000000
25%,NaN,5.000000
50%,NaN,5.000000
75%,NaN,5.000000


In [20]:
test_sub.to_csv("Submissions/First.csv", index=False)

In [21]:
test_sub['rating_score'].value_counts()

5    44913
1     3964
4      164
2       23
3       15
Name: rating_score, dtype: int64